In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import os
from os import listdir
import numpy as np
from numpy import asarray
from numpy import vstack
from numpy import expand_dims
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import save_img
from numpy import savez_compressed

from numpy import load
from numpy import ones, zeros
from numpy.random import randint
from matplotlib import pyplot

In [3]:
import PIL
from PIL import Image

In [4]:
from numpy import load
from numpy import ones, zeros
from numpy.random import randint
from matplotlib import pyplot

# Build Model

In [5]:
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, LeakyReLU, BatchNormalization, Activation, Concatenate, Conv2DTranspose, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
def define_deskriminator(des_shape):
    init = RandomNormal(stddev=0.02)
    in_src_images = Input(shape=des_shape)
    in_target_images = Input(shape=des_shape)
    #Merge images
    merged = Concatenate()([in_src_images, in_target_images])
    #Convolusi
    #C64
    C = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    C = LeakyReLU(alpha=0.2)(C)
    #C128
    C = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(C)
    C = BatchNormalization()(C)
    C = LeakyReLU(alpha=0.2)(C)
    #C256
    C = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(C)
    C = BatchNormalization()(C)
    C = LeakyReLU(alpha=0.2)(C)
    #C512
    C = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(C)
    C = BatchNormalization()(C)
    C = LeakyReLU(alpha=0.2)(C)
    #C512
    C = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(C)
    C = BatchNormalization()(C)
    C = LeakyReLU(alpha=0.2)(C)
    #Output
    C = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(C)
    output = Activation('sigmoid')(C)
    
    modeld = Model([in_src_images, in_target_images], output)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    modeld.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    
    return modeld

In [ ]:
#Define Encoder Block
def define_encoder(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    #downsampling Layers
    e = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        e = BatchNormalization()(e, training=True)
    e = LeakyReLU(alpha=0.2)(e)
    
    return e

#Define Decoder Block
def define_decoder(layer_in, skip_in, n_filters, dropout=True):
    init =RandomNormal(stddev=0.02)
    
    d = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    d = BatchNormalization()(d, training=True)
    if dropout:
        d = Dropout(0.5)(d, training=True)
    d = Concatenate()([d, skip_in])
    d = Activation('relu')(d)
    return d

In [ ]:
def define_generator(image_shape=(256,256,3), sar_shape=(256,256,1)):
    init = RandomNormal(stddev=0.02)
    in_img_gen = Input(shape=image_shape)
    in_sar_gen = Input(shape=sar_shape)
    
    #Merge images
    cat = Concatenate()([in_img_gen, in_sar_gen])
    #Encoder
    e1 = define_encoder(cat, 64, batchnorm=False)
    e2 = define_encoder(e1, 128)
    e3 = define_encoder(e2, 256)
    e4 = define_encoder(e3, 512)
    e5 = define_encoder(e4, 512)
    e6 = define_encoder(e5, 512)
    e7 = define_encoder(e6, 512)
    
    #Bottleneck
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    
    d1 = define_decoder(b, e7, 512)
    d2 = define_decoder(d1, e6, 512)
    d3 = define_decoder(d2, e5, 512)
    d4 = define_decoder(d3, e4, 512, dropout=False)
    d5 = define_decoder(d4, e3, 256, dropout=False)
    d6 = define_decoder(d5, e2, 128, dropout=False)
    d7 = define_decoder(d6, e1, 64, dropout=False)
    
    #output
    g = Conv2DTranspose(4, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_img = Activation('tanh')(g)
    
    modelg = Model([in_img_gen, in_sar_gen], out_img)
    return modelg

In [ ]:
def define_gan(g_model, d_model, image_shape, sar_shape):
    #Membuat deskriminator not Trainable
    d_model.trainable = False
    #membuat variable source gambar
    gen_in_img = Input(shape=image_shape)
    gen_in_sar = Input(shape=sar_shape)
    #membuat output generator
    gen_out = g_model([gen_in_img, gen_in_sar])
    #Membuat input gambar dan output generator sebagai input deskriminator
    dis_out = d_model([gen_in_img, gen_out])
    #membuat gan model
    gan_model = Model([gen_in_img, gen_in_sar], [dis_out, gen_out])
    
    opt = Adam(lr=0.0002, beta_1=0.5)
    gan_model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    
    return gan_model
    

In [ ]:
des = define_deskriminator((256,256,4))
des.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 4) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 256, 256, 4) 0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256, 256, 8)  0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 8256        concatenate[0][0]            

In [ ]:
gen = define_generator((256,256,4), (256,256,1))
gen.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 4) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 256, 256, 5)  0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 128, 128, 64) 5184        concatenate_1[0][0]        

In [ ]:
gan = define_gan(gen, des, (256,256,4), (256,256,1))
gan.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 256, 4) 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 256, 256, 4)  54433412    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
model (Model)                   (None, 16, 16, 1)    6970305     input_5[0][0]              

# LOAD SAMPLE AND LABEL

In [7]:
def load_scaling_sample(path):
    data = load(path)
    X1, X2, X3 = data['arr_0'], data['arr_1'], data['arr_2']
    #scaling format [0,255] ke [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    X3 = (X3 - 127.5) / 127.5
    return [X1, X2, X3]

In [ ]:
dataset = load_scaling_sample('/content/drive/My Drive/SARGAN/SAROptGAN_datasets.npz')

In [ ]:
def generate_real_sample(dataset, n_samples, patch_shape):
    trainA, trainB, trainC = dataset
    #memilih random samples
    ix = randint(0, trainA.shape[0], n_samples)
    #retrieve selected image
    X1, X2, X3 = trainA[ix], trainB[ix], trainC[ix]
    #Labeling 'real' class to 1
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return X1, X2, X3, y

In [ ]:
def generate_fake_sample(g_model, dataset_cloud, dataset_sar, patch_shape):
    #generate fake images
    X = g_model.predict([dataset_cloud, dataset_sar])
    #Labeling fake class to 0
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [ ]:
def summary_peformance(step, g_model, dataset, n_samples=3):
    #Generate Xreal dan Xfake samples
    X_realA, X_realB, X_realC, _ = generate_real_sample(dataset, n_samples, 1)
    X_fakeB, _ = generate_fake_sample(g_model, X_realA, X_realB, 1)

    #Scaling (0,1)
    X_realA = (X_realA + 1)/2
    X_realC = (X_realC + 1)/2
    X_fakeB = (X_fakeB + 1)/2
    
    #Ploing source images, generate image, target image
    #plot images
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, i+1)
        pyplot.axis('off')
        pyplot.imshow(X_realA[i])
    #Plot Hasil Generate
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, i+ n_samples + 1)
        pyplot.axis('off')
        pyplot.imshow(X_fakeB[i])
    #Plot Target Image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, i+ n_samples*2 + 1)
        pyplot.axis('off')
        pyplot.imshow(X_realC[i])
     
    #Save Plot Images
    filename1 = '/content/drive/My Drive/SARGAN/SAROptGAN/images/plot_%06d.png' % (step+1)
    pyplot.savefig(filename1, dpi=200)
    pyplot.close()
    #save generator model
    filename2 = '/content/drive/My Drive/SARGAN/SAROptGAN/models/model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [ ]:
dloss_real = []
dloss_fake = []
gan_loss = []

#TRAIN

In [ ]:
#training image to image translation model
def train(g_model, d_model, gan_model, dataset, n_epochs=300, n_batch=4):
    #Menentukan output square Deskriminator
    n_path = d_model.output_shape[1]
    #Unpacking dataset
    trainA, trainB, trainC = dataset
    #Menentukan batch per epoch
    batch_per_epoch = int(len(trainA)/n_batch)
    #Menentukan jumlah steps
    n_steps = batch_per_epoch * n_epochs
    
    for i in range(n_steps):
        #Select real samples dalam batch
        X_realA, X_realB, X_realC, y_real = generate_real_sample(dataset, n_batch, n_path)
        #Generate Fake samples
        X_fakeB, y_fake = generate_fake_sample(g_model, X_realA, X_realB, n_path)
        #Train Diskiminator untuk real sample
        d_loss1 = d_model.train_on_batch([X_realA, X_realC], y_real)
        #Train Diskriminator untuk fake sample
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        #Train Generator
        g_loss, _, _ = gan_model.train_on_batch([X_realA, X_realB], [y_real, X_realC])
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        
        dloss_real.append(d_loss1)
        dloss_fake.append(d_loss2)
        gan_loss.append(g_loss)
        #Print and save peformance
        if (i+1) % batch_per_epoch == 0:
            summary_peformance(i, g_model, dataset)
        

In [ ]:
with tf.device('/device:GPU:0'):
    train(gen, des, gan, dataset, 300, 32)

Streaming output truncated to the last 5000 lines.
>454, d1[0.014] d2[0.014] g[9.962]
>455, d1[0.033] d2[0.015] g[7.669]
>456, d1[0.009] d2[0.011] g[9.364]
>457, d1[0.026] d2[0.007] g[8.577]
>458, d1[0.021] d2[0.009] g[8.173]
>459, d1[0.024] d2[0.004] g[7.810]
>460, d1[0.007] d2[0.005] g[8.252]
>461, d1[0.031] d2[0.015] g[6.952]
>462, d1[0.153] d2[0.014] g[7.583]
>463, d1[0.002] d2[1.058] g[10.619]
>464, d1[0.109] d2[0.004] g[9.658]
>465, d1[0.574] d2[0.200] g[7.617]
>466, d1[0.041] d2[0.632] g[9.358]
>467, d1[0.144] d2[0.051] g[7.246]
>468, d1[0.027] d2[0.082] g[9.888]
>469, d1[0.069] d2[0.011] g[8.279]
>470, d1[0.026] d2[0.010] g[8.409]
>471, d1[0.182] d2[0.024] g[6.517]
>472, d1[0.012] d2[0.035] g[9.314]
>473, d1[0.038] d2[0.018] g[6.953]
>474, d1[0.028] d2[0.010] g[7.281]
>Saved: /content/drive/My Drive/SARGAN/SAROptGAN/images/plot_000474.png and /content/drive/My Drive/SARGAN/SAROptGAN/models/model_000474.h5
>475, d1[0.009] d2[0.008] g[7.581]
>476, d1[0.010] d2[0.009] g[7.629]
>47

## #PREDICT

In [7]:
#Predict
#Load Model
model = load_model('/content/drive/My Drive/SARGAN/SAROptGAN/model_011850.h5')

In [8]:
def load_predict(path):
    img = Image.open(path)
    img = img_to_array(img)
    img = (img - 127.5) / 127.5
    img = expand_dims(img, 0)
    return img

In [9]:
cloud = load_predict('/content/drive/My Drive/SARGAN/SAROptGAN/cloud_11.tif')
SAR = load_predict('/content/drive/My Drive/SARGAN/SAROptGAN/SAR_11.tif')

In [15]:
def translate_image(model, imagecloud, imagesar, chip_dim=256):
    images = np.empty((imagecloud.shape[1],imagecloud.shape[2],imagecloud.shape[3]))
    for row in range(0,imagecloud.shape[1],chip_dim):
        for col in range(0, imagecloud.shape[2], chip_dim):
            result = model.predict([imagecloud[:, row: row+chip_dim, col: col+chip_dim, :], imagesar[:, row: row+chip_dim, col: col+chip_dim, :]])
            images[row: row+chip_dim, col: col+chip_dim, :]=result[0]
    return images

In [32]:
hasil = translate_image(model, cloud, SAR, 256)

In [34]:
save_img('/content/drive/My Drive/SARGAN/SAROptGAN/Hasil_11_5.tif', hasil)

In [33]:
for i in range(0,3):
  Ms = MinMaxScaler(feature_range=(0,255))
  Ms_fit = Ms.fit(hasil[:,:,i])
  hasil[:,:,i] = Ms_fit.transform(hasil[:,:,i])